## Import Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import os
import time
import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from sklearn import metrics
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.utils.multiclass import type_of_target
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

from PIL import Image
from osgeo import gdal

Mounted at /content/drive


## Data Exploration


In [ ]:
bc_data = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/BC_input_data.CSV'
df_bc = pd.read_csv(bc_data)

print(df_bc.info())
df_bc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112785 entries, 0 to 5112784
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   B01         float64
 1   B02         float64
 2   B03         float64
 3   B04         float64
 4   B05         float64
 5   B06         float64
 6   B07         float64
 7   B08         float64
 8   B09         float64
 9   B11         float64
 10  B12         float64
 11  B8A         float64
 12  land_cover  int64  
dtypes: float64(12), int64(1)
memory usage: 507.1 MB
None


,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,land_cover
0,0.0172,0.0200,0.0373,0.0243,0.0643,0.1630,0.1883,0.1995,0.2100,0.1222,0.0642,0.2045,14
1,0.0161,0.0297,0.0476,0.0447,0.0834,0.1770,0.2067,0.2224,0.2265,0.1829,0.1020,0.2302,14
2,0.0140,0.0151,0.0249,0.0241,0.0459,0.0992,0.1188,0.1263,0.2059,0.1082,0.0585,0.1328,14
3,0.0132,0.0267,0.0435,0.0411,0.0794,0.1701,0.1955,0.2137,0.2261,0.1669,0.0943,0.2197,18
4,0.0142,0.0255,0.0422,0.0389,0.0804,0.1700,0.1958,0.2122,0.2349,0.1546,0.0848,0.2189,18
5,0.0149,0.0261,0.0480,0.0372,0.0959,0.2127,0.2442,0.2639,0.2662,0.1509,0.0806,0.2694,18
6,0.0217,0.0289,0.0481,0.0379,0.0863,0.1801,0.2070,0.2180,0.2344,0.1499,0.0834,0.2239,14
7,0.0278,0.0495,0.0721,0.0628,0.1189,0.2124,0.2368,0.2499,0.2588,0.1794,0.1022,0.2572,14
8,0.0153,0.0172,0.0306,0.0266,0.0602,0.1190,0.1369,0.1463,0.1926,0.0895,0.0486,0.1509,2
9,0.0069,0.0066,0.0119,0.0145,0.0233,0.0281,0.0348,0.0316,0.0949,0.0295,0.0158,0.0360,18


In [ ]:
on_data = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/ON_input_data.CSV'
df_on = pd.read_csv(on_data)

print(df_on.info())
df_on.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112785 entries, 0 to 5112784
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   B01         float64
 1   B02         float64
 2   B03         float64
 3   B04         float64
 4   B05         float64
 5   B06         float64
 6   B07         float64
 7   B08         float64
 8   B09         float64
 9   B11         float64
 10  B12         float64
 11  B8A         float64
 12  land_cover  int64  
dtypes: float64(12), int64(1)
memory usage: 507.1 MB
None


,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,land_cover
0,0.0172,0.0200,0.0373,0.0243,0.0643,0.1630,0.1883,0.1995,0.2100,0.1222,0.0642,0.2045,14
1,0.0161,0.0297,0.0476,0.0447,0.0834,0.1770,0.2067,0.2224,0.2265,0.1829,0.1020,0.2302,14
2,0.0140,0.0151,0.0249,0.0241,0.0459,0.0992,0.1188,0.1263,0.2059,0.1082,0.0585,0.1328,14
3,0.0132,0.0267,0.0435,0.0411,0.0794,0.1701,0.1955,0.2137,0.2261,0.1669,0.0943,0.2197,18
4,0.0142,0.0255,0.0422,0.0389,0.0804,0.1700,0.1958,0.2122,0.2349,0.1546,0.0848,0.2189,18
5,0.0149,0.0261,0.0480,0.0372,0.0959,0.2127,0.2442,0.2639,0.2662,0.1509,0.0806,0.2694,18
6,0.0217,0.0289,0.0481,0.0379,0.0863,0.1801,0.2070,0.2180,0.2344,0.1499,0.0834,0.2239,14
7,0.0278,0.0495,0.0721,0.0628,0.1189,0.2124,0.2368,0.2499,0.2588,0.1794,0.1022,0.2572,14
8,0.0153,0.0172,0.0306,0.0266,0.0602,0.1190,0.1369,0.1463,0.1926,0.0895,0.0486,0.1509,2
9,0.0069,0.0066,0.0119,0.0145,0.0233,0.0281,0.0348,0.0316,0.0949,0.0295,0.0158,0.0360,18


#### CNN Analysis

In [ ]:
#load dataset in train/test variables
def load_dataset(df_train, df_test, num_classes):
    
    scaler = StandardScaler()

    df_train = np.array(df_train)    
    # df_train = scaler.fit_transform(df_train)
    df_train = df_train.reshape( len(df_train), 1, -1 )
    trainX, trainy = df_train[:,:,0:-1], to_categorical(df_train[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    df_test = np.array(df_test)
    # df_test = scaler.fit_transform(df_test)
    df_test = df_test.reshape( len(df_test), 1, -1 )
    testX, testy = df_test[:,:,0:-1], to_categorical(df_test[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 10000
    dropout, recurrent_dropout = 0.5, 0.5
    n_features, n_outputs = trainX.shape[2], trainy.shape[-1]
    model = models.Sequential()
    model.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, n_features)))
    model.add(layers.Conv1D(filters=64, kernel_size=1, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.MaxPooling1D(pool_size=1))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return model, accuracy
       
    
# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset(df_on, df_bc, num_classes=20)  
    
    # repeat experiment
    scores = list()
    for r in range(repeats):
        model, score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    print(model)
    # summarize results
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

    return model

st = time.time()
model = run_experiment(repeats=2)
print(model)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

# Save the model as pickle file

pickle_path = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/model_cnn.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)

Epoch 1/10
512/512 [==============================] - 33s 62ms/step - loss: 1.2811 - accuracy: 0.5687
Epoch 2/10
512/512 [==============================] - 30s 58ms/step - loss: 0.9728 - accuracy: 0.6477
Epoch 3/10
512/512 [==============================] - 30s 58ms/step - loss: 0.9486 - accuracy: 0.6540
Epoch 4/10
512/512 [==============================] - 31s 61ms/step - loss: 0.9358 - accuracy: 0.6573
Epoch 5/10
512/512 [==============================] - 30s 59ms/step - loss: 0.9263 - accuracy: 0.6595
Epoch 6/10
512/512 [==============================] - 30s 59ms/step - loss: 0.9198 - accuracy: 0.6612
Epoch 7/10
512/512 [==============================] - 30s 59ms/step - loss: 0.9149 - accuracy: 0.6625
Epoch 8/10
512/512 [==============================] - 30s 59ms/step - loss: 0.9117 - accuracy: 0.6633
Epoch 9/10
512/512 [==============================] - 30s 59ms/step - loss: 0.9088 - accuracy: 0.6642
Epoch 10/10
512/512 [==============================] - 30s 59ms/step - loss: 0.906

In [ ]:
#### Load in your model for testing here
tk_pickle_path = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/model_cnn.pkl'

YOUR_MODEL_PATH = tk_pickle_path

clf = pickle.load(open(YOUR_MODEL_PATH, 'rb'))
print(clf)

In [ ]:
X, y, _, _ = load_dataset(df_on, df_bc, num_classes=20)
y = np.argmax(y, axis=1)

In [ ]:
#### Predict and check skill
y_pred =  np.argmax(clf.predict(X), axis=1)

print(f"Accuracy: {metrics.accuracy_score(y, y_pred):0.4f}")

Accuracy: 0.6061


CNN Trials

In [ ]:
#load dataset in train/test variables
def load_dataset(df_train, df_test, num_classes):
    
    df_train = np.array(df_train).reshape( len(df_train), 1, -1 )
    trainX, trainy = df_train[:,:,0:-1], to_categorical(df_train[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    df_test = np.array(df_test).reshape( len(df_test), 1, -1 )
    testX, testy = df_test[:,:,0:-1], to_categorical(df_test[:,:,-1], num_classes=num_classes)  # one hot encode y
    
    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 10000
    dropout, recurrent_dropout, lstm_units = 0.5, 0.5, 100
    n_features, n_outputs = trainX.shape[2], trainy.shape[-1]
    model = models.Sequential()
    model.add(layers.Conv1D(filters=10, kernel_size=1, activation='relu', input_shape=(1, n_features)))
    model.add(layers.Conv1D(filters=20, kernel_size=1, activation='relu'))
    model.add(layers.Conv1D(filters=30, kernel_size=1, activation='relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Masking(mask_value=0.0))
    model.add(layers.LSTM(lstm_units, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True))
    # model.add(layers.Bidirectional(layers.LSTM(lstm_units, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True)))
    # model.add(layers.Bidirectional(layers.LSTM(lstm_units, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True)))
    # model.add(layers.Bidirectional(layers.LSTM(lstm_units, dropout=dropout, recurrent_dropout=recurrent_dropout)))
   
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return model, accuracy
       
    
# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset(df_on, df_bc, num_classes=20)  
    
    # repeat experiment
    scores = list()
    for r in range(repeats):
        model, score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    print(model)
    # summarize results
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

    return model

st = time.time()
model = run_experiment(repeats=2)
print(model)
print( f'Total Elapsed time = {(time.time() - st)/60 :.1f} min')

# Save the model as pickle file

pickle_path = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/model_conv1D_LSTM.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)

Epoch 1/10
512/512 [==============================] - 147s 279ms/step - loss: 1.4966 - accuracy: 0.5001
Epoch 2/10
512/512 [==============================] - 141s 275ms/step - loss: 1.0922 - accuracy: 0.6173
Epoch 3/10
512/512 [==============================] - 143s 278ms/step - loss: 1.0247 - accuracy: 0.6363
Epoch 4/10
512/512 [==============================] - 142s 278ms/step - loss: 0.9973 - accuracy: 0.6429
Epoch 5/10
512/512 [==============================] - 143s 280ms/step - loss: 0.9777 - accuracy: 0.6471
Epoch 6/10
512/512 [==============================] - 142s 277ms/step - loss: 0.9662 - accuracy: 0.6499
Epoch 7/10
512/512 [==============================] - 142s 277ms/step - loss: 0.9584 - accuracy: 0.6516
Epoch 8/10
512/512 [==============================] - 142s 278ms/step - loss: 0.9527 - accuracy: 0.6529
Epoch 9/10
512/512 [==============================] - 143s 280ms/step - loss: 0.9478 - accuracy: 0.6541
Epoch 10/10
512/512 [==============================] - 141s 276m

In [ ]:
# Save the model as pickle file

pickle_path = r'/content/drive/MyDrive/Colab Notebooks/AISC/SentinelAI/model_conv1D_LSTM.pkl'
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)

FileNotFoundError: ignored

In [ ]:
#No pooling + filter(32) => accuracy is 49%
#No pooling + filter(10,20,30) => accuracy is 51%
#No pooling + Conv1D + LSTM => accuracy is 51%